In [2]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/spaceus9/mlops-datascience.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="spaceus9"
os.environ["MLFLOW_TRACKING_PASSWORD"]="0179d1381043c9c2e57e8a8b46f61db36809b7b2"


In [3]:
import os
%pwd

'c:\\Users\\pc\\Desktop\\ai\\mlops-datascience\\research'

In [5]:
os.chdir("../")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path:Path
    model_path: Path
    all_params: Path
    metric_file_name: Path
    target_column: str
    mlflow_url: str

In [7]:
from src.mlops_pipeline.constants import *
from src.mlops_pipeline.utils.common import read_yaml,create_directories,save_json

In [9]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml([config_filepath])
        self.params = read_yaml([params_filepath])
        self.schema = read_yaml([schema_filepath])
        
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_url="https://dagshub.com/spaceus9/mlops-datascience.mlflow"
            
            
            
        )
        
        return model_evaluation_config

In [11]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [13]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config
        
    def eval_metrices(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae= mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2
    
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x= test.data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse,mae,r2) = self.eval_metrices(test_y,predicted_qualities)
            
            scores = {"rmse":rmse, "mae":mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("r2",r2)
            mlflow.log_metric("mae",mae)
            
            if tracking_url_type_store !="file":
                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model,"model")
            
            
            

In [ ]:
try:
    config= ConfigurationManager()
    data_evaluation_config= config.get_model_evaluation_config()
    data_evaluation = ModelEvaluation(config=data_evaluation_config)
    data_evaluation.log_into_mlflow()
except Exception as e:
    raise e